In [1]:
import sys
import pathlib
import os

def find_root(path):
    if os.path.split(path)[-1] != "amazon-routing-challenge":
        return find_root(os.path.split(path)[0])
    return path


ROOT = find_root(pathlib.Path().absolute())
sys.path.append(ROOT)


In [2]:

import json
import pandas as pd
import plotly.graph_objects as go
from dotenv import load_dotenv

_ = load_dotenv()


In [3]:
data_path = os.path.join(
    ROOT,
    "data",
    "almrrc2021-data-training",
    "model_build_inputs",
)


## Open the Package Data

In [4]:
with open(os.path.join(data_path, "package_data.json"), 'r') as f:
    package_data = json.load(f)

## Create the Route DataFrame


In [5]:
with open(os.path.join(data_path, "route_data.json"), "r") as f:
    route_data = json.load(f)


In [6]:
len(route_data)

6112

In [7]:
route_df = pd.DataFrame.from_records(
    (
        {
            "stop_id": k,
            "route_id": route_id,
            "station_code": route_data[route_id]["station_code"],
            "departure_datetime": route_data[route_id]["date_YYYY_MM_DD"] + " " + route_data[route_id]["departure_time_utc"],
            "executor_capacity_cm3": route_data[route_id]["executor_capacity_cm3"],
            "route_score": route_data[route_id]["route_score"],
            **v,
        }
        for route_id in route_data.keys()
        for k, v in route_data[route_id]["stops"].items()
    )
)


## Open the Sequence Data

In [8]:
with open(os.path.join(data_path, "actual_sequences.json"), "r") as f:
    actual_sequences = json.load(f)

In [9]:
sequential_pairs = {}

for k in actual_sequences.keys():
    _t = [v[0] for v in sorted(tuple(actual_sequences[k]['actual'].items()), key=lambda x: x[1])]
    sequential_pairs[k] = tuple(zip(_t, _t[1:]))

## Investigate the Stations


In [10]:
route_df["station_code"].unique()


array(['DLA3', 'DSE4', 'DSE5', 'DLA9', 'DLA7', 'DCH4', 'DBO2', 'DBO3',
       'DLA8', 'DLA5', 'DCH3', 'DCH1', 'DAU1', 'DCH2', 'DLA4', 'DSE2',
       'DBO1'], dtype=object)

## Number of Routes Missing Zone ID


All papers (except 1) solve by replacing missing Zone ID with euclidean distance nearest zone id


In [11]:
bad_routes = route_df.loc[
    ((route_df.zone_id.isna()) & (route_df.type != "Station"))
    | (route_df.lat.isna() | route_df.lng.isna())
    # | (route_df.route_score != "High")
].route_id.unique()
len(bad_routes)


2544

In [12]:
good_route_df = route_df.copy() # loc[~route_df.route_id.isin(bad_routes)].copy()
len(good_route_df.route_id.unique())

6112

In [13]:
_dup_stops = good_route_df.groupby(["lat", "lng"]).agg({"route_id": "nunique"})
_dup_stops.head()

,,route_id
lat,lng,
30.117288,-97.968165,1
30.118834,-97.988569,1
30.119146,-97.983968,1
30.120233,-97.983939,1
30.120723,-97.983840,1


In [14]:
_dup_stops.loc[(_dup_stops.route_id > 1)]

,,route_id
lat,lng,
30.129676,-97.963750,2
30.132366,-97.965042,2
30.136697,-97.960858,3
30.136878,-97.968069,2
30.137279,-97.961708,2
...,...,...
48.125239,-122.157453,2
48.125886,-122.143153,2
48.127115,-122.155157,2


### Calculate the Datetime Specifics

In [15]:
good_route_df.departure_datetime = pd.to_datetime(good_route_df.departure_datetime)
# good_route_df.sort_values(by="departure_datetime", inplace=True)
good_route_df.departure_datetime.min(), good_route_df.departure_datetime.max()

(Timestamp('2018-07-19 13:41:42'), Timestamp('2018-08-26 16:14:30'))

### Plotting where the Stations are at


In [16]:
fig = go.Figure()

_filtered_df = good_route_df.loc[good_route_df["type"] == "Station"]


for station_code in _filtered_df["station_code"].unique():
    _df = _filtered_df.loc[_filtered_df["station_code"] == station_code]
    fig.add_trace(
        go.Scattermapbox(
            name=station_code,
            lat=[_df["lat"].mean()],
            lon=[_df["lng"].mean()],
            mode="markers",
            marker=dict(
                size=10,
            ),
            text=_df["station_code"],
            textfont=dict(
                family="sans serif",
                size=22,
            ),
        )
    )

fig.update_layout(
    # autosize=True,
    height=600,
    width=1000,
    hovermode="closest",
    mapbox=go.layout.Mapbox(
        accesstoken=os.environ["MAPBOX_KEY"],
        style="mapbox://styles/max-schrader/ck8t1cmmc02wk1it9rv28iyte",
        # style="mapbox://styles/max-schrader/cl6lhvrfw001516pkh3s6iv7l",
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=_filtered_df["lat"].mean(), lon=_filtered_df["lng"].mean()
        ),
        pitch=0,
        zoom=2,
    ),
    # margin=go.layout.Margin(l=0, r=0, t=0, b=0),
)


### Creating a Route ID to Station ID Dictionary

In [17]:
station_route_id = dict(good_route_df.loc[good_route_df["type"] == "Station"][["route_id", "stop_id"]].values)

## Investigating Duplicate Dropoffs


In [18]:
good_route_df["duplicates"] = good_route_df.duplicated(
    subset=["lat", "lng"], keep=False
) 


In [19]:
good_route_df["duplicates"] = good_route_df.duplicated(
    subset=["lat", "lng"], keep=False
) #& (good_route_df.type != "Station")


lat_lon_dup = good_route_df.loc[good_route_df.duplicates].groupby(["lat", "lng"])['route_id'].unique().reset_index()
# lat_lon_dup.iloc[10].route_id

In [20]:
lat_lon_dup.route_id = lat_lon_dup.route_id.apply(lambda x: tuple(x))

In [21]:
lat_lon_dup

,lat,lng,route_id
0,30.129676,-97.963750,"(RouteID_9ea87ee2-867b-4cc0-ab72-f76d8389182a,..."
1,30.132366,-97.965042,"(RouteID_5ab24a12-62bf-4b91-9b49-3be9c3509b7b,..."
2,30.136697,-97.960858,"(RouteID_5ab24a12-62bf-4b91-9b49-3be9c3509b7b,..."
3,30.136878,-97.968069,"(RouteID_c8a6bb25-32bd-424a-93eb-46e9261ae1e9,..."
4,30.137279,-97.961708,"(RouteID_9ea87ee2-867b-4cc0-ab72-f76d8389182a,..."
...,...,...,...
129952,48.125239,-122.157453,"(RouteID_5588756f-5963-424d-ada6-cf0fb77c82cd,..."
129953,48.125886,-122.143153,"(RouteID_076b21cf-9d76-4d57-9859-cf3fb783533f,..."
129954,48.127115,-122.155157,"(RouteID_076b21cf-9d76-4d57-9859-cf3fb783533f,..."
129955,48.127870,-122.159378,"(RouteID_5588756f-5963-424d-ada6-cf0fb77c82cd,..."


In [22]:
lat_lon_map = {}

def create_map(row):
    for route_id in row.route_id:
        if route_id not in lat_lon_map:
            lat_lon_map[route_id] = []
        lat_lon_map[route_id].append(tuple(row[['lat', 'lng']]))

_ = lat_lon_dup.apply(
    create_map, axis=1
)

for route_id in lat_lon_map:
    lat_lon_map[route_id] = list(set(lat_lon_map[route_id]))

### Create a DataFrame with the matched pairs

In [23]:
import itertools

In [24]:
match_pairs = []

# this isn't computationaly efficient, but it's a small enough dataset that doesn't matter
covered_pairs = set()
for r_1 in lat_lon_map:
        match_pairs.extend((r_1, *match_location)
                           for match_location in itertools.product(
                               lat_lon_map[r_1], lat_lon_map[r_1])
                           if match_location[0] != match_location[1])
    # lat_lon_map[r_1]

In [25]:
tt_df = pd.DataFrame(match_pairs, columns=["route_id", "from", "to"])
_loc_pairs = tt_df.groupby(["from", "to"]).agg({"route_id": "nunique"}).reset_index()
# keep only the ones that have more than one route
_loc_pairs = _loc_pairs.loc[_loc_pairs.route_id > 1]
_loc_pairs.head()

,from,to,route_id
1,"(30.129676, -97.96375)","(30.136697, -97.960858)",2
2,"(30.129676, -97.96375)","(30.137279, -97.961708)",2
3,"(30.129676, -97.96375)","(30.137574, -97.957124)",2
10,"(30.129676, -97.96375)","(30.152141, -97.95341)",2
18,"(30.129676, -97.96375)","(30.156254, -97.948389)",2


In [26]:
okay_tuples = tuple(tuple(v) for v in _loc_pairs[['to', 'from']].itertuples(index=False))

In [27]:
tt_df = tt_df.loc[tt_df[['from', 'to']].apply(tuple, axis=1).isin(okay_tuples)].reset_index(drop=True)

#### Add Information to the DF

In [28]:
# build a map of id -> lat, lon -> stop_id
lat_lon_2_id = {r_id: {
    (stop_info['lat'], stop_info['lng']): stop_id for stop_id, stop_info in r_data['stops'].items()
} for r_id, r_data in route_data.items()}

# build a map of route_id -> depature_datetime
route_id_2_departure_datetime = good_route_df.groupby('route_id').first()['departure_datetime'].to_dict()

In [29]:
# find the matching stop id for each route
def to_stop_id(row, ):
    return lat_lon_2_id[row[0]].get(row[1])

tt_df['from_id'] = tt_df[['route_id', 'from']].apply(to_stop_id, axis=1, raw=True)
tt_df['to_id'] = tt_df[['route_id', 'to']].apply(to_stop_id, axis=1, raw=True)
tt_df['datetime'] = tt_df['route_id'].apply(lambda x: route_id_2_departure_datetime[x])

In [30]:
tt_df.head()

,route_id,from,to,from_id,to_id,datetime
0,RouteID_9ea87ee2-867b-4cc0-ab72-f76d8389182a,"(30.186342, -97.925433)","(30.178999, -97.924018)",DS,AR,2018-08-12 13:32:04
1,RouteID_9ea87ee2-867b-4cc0-ab72-f76d8389182a,"(30.186342, -97.925433)","(30.184255, -97.919351)",DS,WR,2018-08-12 13:32:04
2,RouteID_9ea87ee2-867b-4cc0-ab72-f76d8389182a,"(30.186342, -97.925433)","(30.186857, -97.923924)",DS,LQ,2018-08-12 13:32:04
3,RouteID_9ea87ee2-867b-4cc0-ab72-f76d8389182a,"(30.186342, -97.925433)","(30.177748, -97.923552)",DS,PT,2018-08-12 13:32:04
4,RouteID_9ea87ee2-867b-4cc0-ab72-f76d8389182a,"(30.186342, -97.925433)","(30.445236, -97.709418)",DS,PZ,2018-08-12 13:32:04


#### Apply Travel Time

In [31]:
tt_json = json.loads(open(os.path.join(data_path, "travel_times.json"), "r").read())


In [32]:

# remove ids that are note in route_location_df
keep_routes = set(tt_df["route_id"].unique())
tt_json = {k: v for k, v in tt_json.items() if k in keep_routes}

In [33]:
def add_travel_time(row,):
    return tt_json[row[0]][row[2]][row[1]]


tt_df['tt'] = tt_df[['route_id', 'to_id', 'from_id']].apply(add_travel_time, axis=1, raw=True).values
# tt_df[('r2', 'tt')] = tt_df[[('route_2', ''), ('r2', "to_id"), ('r2', "from_id")]].apply(add_travel_time, axis=1, raw=True).values
# tt_df[('r1', 'tt)]] = tt_df[[('route_1', ''), ('r1', "to_id"), ('r1', "from_id")]].apply(add_travel_time, axis=1, raw=True)

#### Check Whether From/To actually occured

In [34]:
def check_actual_occurance(row,):
    return (row[1], row[2]) in sequential_pairs[row[0]]

tt_df['actual_occurance'] = tt_df[['route_id', 'from_id', 'to_id']].apply(check_actual_occurance, axis=1, raw=True)

### Calculate the Travel Time Estimate Differences for a give Pair

In [35]:
grouper = tt_df.groupby(['from', 'to'])

In [36]:
grouped_df = grouper.agg({'tt': ('mean', 'std', 'min', 'max'), 'route_id': 'nunique', 'actual_occurance': 'max'}, )
grouped_df.head()

tt                       \
                                                  mean  std     min     max   
from                   to                                                     
(30.129676, -97.96375) (30.136697, -97.960858)   181.4  0.0   181.4   181.4   
                       (30.137279, -97.961708)   195.5  0.0   195.5   195.5   
                       (30.137574, -97.957124)   203.5  0.0   203.5   203.5   
                       (30.152141, -97.95341)   1004.7  0.0  1004.7  1004.7   
                       (30.156254, -97.948389)   881.0  0.0   881.0   881.0   

                                               route_id actual_occurance  
                                                nunique              max  
from                   to                                                 
(30.129676, -97.96375) (30.136697, -97.960858)        2            False  
                       (30.137279, -97.961708)        2            False  
                       (30.137574, -97.957124)        2             True  
                       (30.152141, -97.95341)         2            False  
                       (30.156254, -97.948389)        2            False

In [37]:
grouped_df['tt_diff'] = grouped_df[('tt', 'max')] - grouped_df[('tt', 'min')]

#### Percent of Locations that Have TT Estimate Deviation 

In [38]:
(len(grouped_df.loc[grouped_df[('tt', 'std')] > 0]) / len(grouped_df)) * 100

9.869566177967766

#### Percent of Location Pairs that Actually Occur

In [39]:
sum(grouped_df[('actual_occurance', 'max')]),  len(grouped_df), (sum(grouped_df[('actual_occurance', 'max')]) / len(grouped_df)) * 100

(47226, 3349438, 1.4099678811788725)

In [40]:
slicer = grouped_df[('tt', 'std')] > 0

sum(grouped_df.loc[slicer, ('actual_occurance', 'max')]),  len(grouped_df.loc[slicer]), (sum(grouped_df.loc[slicer, ('actual_occurance', 'max')]) / len(grouped_df.loc[slicer])) * 100

(4901, 330575, 1.4825682522876806)

#### Looking at Location Pairs that Actually Occur

In [41]:
grouped_df.loc[grouped_df[('actual_occurance', 'max')]].sort_values(('route_id', 'nunique'), ascending=False)

tt             \
                                                       mean        std   
from                     to                                              
(47.937344, -122.244952) (47.661534, -122.32403)   2150.025  10.522896   
(47.464945, -122.231073) (47.517331, -122.367252)  1500.700   0.000000   
(33.965477, -117.653303) (33.834075, -117.90135)   2320.200   0.000000   
(47.937344, -122.244952) (47.661825, -122.294561)  2519.100   0.000000   
(33.965477, -117.653303) (33.891214, -117.509918)  1623.700   0.000000   
...                                                     ...        ...   
(34.023073, -117.87946)  (34.023937, -117.879214)    14.500   0.000000   
(34.023079, -118.425881) (34.022273, -118.424621)    76.600   0.000000   
                         (34.022626, -118.424898)    72.800   0.000000   
(34.023101, -117.874969) (34.023668, -117.875301)    14.100   0.000000   
(48.124759, -122.162626) (48.124106, -122.161774)    69.400   0.000000   

                                                                  route_id  \
                                                      min     max  nunique   
from                     to                                                  
(47.937344, -122.244952) (47.661534, -122.32403)   2145.6  2184.6       16   
(47.464945, -122.231073) (47.517331, -122.367252)  1500.7  1500.7       11   
(33.965477, -117.653303) (33.834075, -117.90135)   2320.2  2320.2       10   
(47.937344, -122.244952) (47.661825, -122.294561)  2519.1  2519.1       10   
(33.965477, -117.653303) (33.891214, -117.509918)  1623.7  1623.7        9   
...                                                   ...     ...      ...   
(34.023073, -117.87946)  (34.023937, -117.879214)    14.5    14.5        2   
(34.023079, -118.425881) (34.022273, -118.424621)    76.6    76.6        2   
                         (34.022626, -118.424898)    72.8    72.8        2   
(34.023101, -117.874969) (34.023668, -117.875301)    14.1    14.1        2   
(48.124759, -122.162626) (48.124106, -122.161774)    69.4    69.4        2   

                                                  actual_occurance tt_diff  
                                                               max          
from                     to                                                 
(47.937344, -122.244952) (47.661534, -122.32403)              True    39.0  
(47.464945, -122.231073) (47.517331, -122.367252)             True     0.0  
(33.965477, -117.653303) (33.834075, -117.90135)              True     0.0  
(47.937344, -122.244952) (47.661825, -122.294561)             True     0.0  
(33.965477, -117.653303) (33.891214, -117.509918)             True     0.0  
...                                                            ...     ...  
(34.023073, -117.87946)  (34.023937, -117.879214)             True     0.0  
(34.023079, -118.425881) (34.022273, -118.424621)             True     0.0  
                         (34.022626, -118.424898)             True     0.0  
(34.023101, -117.874969) (34.023668, -117.875301)             True     0.0  
(48.124759, -122.162626) (48.124106, -122.161774)             True     0.0  

[47226 rows x 7 columns]

In [42]:
def sort_tt_df_by_location_pair(pair_1, pair_2):
    return tt_df.loc[(tt_df['from'] == pair_1) & (tt_df['to'] == pair_2)]

In [50]:
sort_tt_df_by_location_pair((47.937344, -122.244952),	(47.661534, -122.32403)).sort_values('datetime')

,route_id,from,to,from_id,to_id,datetime,tt,actual_occurance
6267591,RouteID_eb266843-1320-45e3-94fc-784a027a6bf8,"(47.937344, -122.244952)","(47.661534, -122.32403)",RE,GQ,2018-07-20 15:30:57,2145.6,False
6303380,RouteID_e6687a05-2453-4edc-b86c-7558ab6d93f6,"(47.937344, -122.244952)","(47.661534, -122.32403)",GM,DR,2018-07-25 15:10:55,2145.6,False
6269221,RouteID_bcb1dd8c-6e7a-4f1c-8546-933131335d69,"(47.937344, -122.244952)","(47.661534, -122.32403)",BO,JD,2018-07-26 14:51:20,2147.5,False
6266606,RouteID_d73e07a2-4a0a-4457-aea5-96114e36e318,"(47.937344, -122.244952)","(47.661534, -122.32403)",EA,MQ,2018-07-27 15:46:45,2145.6,False
6251160,RouteID_b9825940-2bbf-4356-8d61-1d1339c9070c,"(47.937344, -122.244952)","(47.661534, -122.32403)",WQ,YX,2018-07-29 15:53:52,2145.6,False
6249677,RouteID_40df29e4-d5a3-4f3c-adc0-04465a890696,"(47.937344, -122.244952)","(47.661534, -122.32403)",DY,QU,2018-07-31 16:00:00,2145.6,True
6276529,RouteID_349fe1a3-b174-415c-bf90-d3218c0afc30,"(47.937344, -122.244952)","(47.661534, -122.32403)",RG,DS,2018-08-04 14:48:57,2145.6,False
6263172,RouteID_4ee6e0c9-a4e9-4e61-8d09-e271daeaa28f,"(47.937344, -122.244952)","(47.661534, -122.32403)",VF,PI,2018-08-09 15:17:52,2145.6,False
6261162,RouteID_2fbce317-4620-4360-8e71-3a69e00d3823,"(47.937344, -122.244952)","(47.661534, -122.32403)",TS,UM,2018-08-11 16:00:00,2161.7,False
6272239,RouteID_06751fe1-263c-441a-b482-f7c5d8295c39,"(47.937344, -122.244952)","(47.661534, -122.32403)",AU,LX,2018-08-13 15:49:43,2145.6,False


In [44]:
actual_sequences['RouteID_35222ae6-1540-4b5c-ae35-20e9db557acd']['actual']['BB'], actual_sequences['RouteID_35222ae6-1540-4b5c-ae35-20e9db557acd']['actual']['EO']

(0, 1)

### Show the Location Pairs with Highest Amount of Difference 

In [45]:
grouped_df.loc[grouped_df[('tt', 'std')] > 0, ].sort_values(('tt_diff'), ascending=False).head()

tt                     \
                                                mean         std    min   
from                    to                                                
(42.184325, -71.288743) (42.16578, -71.36275)  559.4  541.078109  176.8   
(42.166706, -71.36514)  (42.16578, -71.36275)  432.7  484.085302   90.4   
(42.173143, -71.60861)  (42.1247, -71.610638)  441.5  472.913015  107.1   
(42.173863, -71.610895) (42.1247, -71.610638)  438.0  472.913015  103.6   
(42.165268, -71.363573) (42.16578, -71.36275)  409.1  471.923066   75.4   

                                                     route_id  \
                                                 max  nunique   
from                    to                                      
(42.184325, -71.288743) (42.16578, -71.36275)  942.0        2   
(42.166706, -71.36514)  (42.16578, -71.36275)  775.0        2   
(42.173143, -71.60861)  (42.1247, -71.610638)  775.9        2   
(42.173863, -71.610895) (42.1247, -71.610638)  772.4        2   
(42.165268, -71.363573) (42.16578, -71.36275)  742.8        2   

                                              actual_occurance tt_diff  
                                                           max          
from                    to                                              
(42.184325, -71.288743) (42.16578, -71.36275)            False   765.2  
(42.166706, -71.36514)  (42.16578, -71.36275)            False   684.6  
(42.173143, -71.60861)  (42.1247, -71.610638)            False   668.8  
(42.173863, -71.610895) (42.1247, -71.610638)            False   668.8  
(42.165268, -71.363573) (42.16578, -71.36275)            False   667.4

### Plotting the Data

In [46]:
import openrouteservice as ors

ors_client = ors.Client(key=os.environ["ORS_KEY"])


def get_ors_travel_time(locations, destination_indexes):
    return ors_client.distance_matrix(
        locations,
        destinations=destination_indexes,
        profile="driving-hgv",
        metrics=["duration", "distance"],
    )


In [47]:
# _r[("to", "")][::-1], _r[("from", "")][::-1]
# https://www.google.com/maps/dir/30.185226,+-97.948886/30.161271,+-97.9378/@30.1844003,-97.954117,3824m/data=!3m1!1e3!4m14!4m13!1m5!1m1!1s0x0:0x58774e0e18241f40!2m2!1d-97.948886!2d30.185226!1m5!1m1!1s0x0:0xc8a5fe56814b73e4!2m2!1d-97.9378!2d30.161271!3e0

In [48]:

# def make_fig(trace):

#     fig = go.Figure()

#     fig.update_layout(
#         # autosize=True,
#         # showlegend=False,
#         height=600,
#         width=1000,
#         hovermode="closest",
#         mapbox=go.layout.Mapbox(
#             accesstoken=os.environ["MAPBOX_KEY"],
#             # style="mapbox://styles/max-schrader/ck8t1cmmc02wk1it9rv28iyte",
#             style="mapbox://styles/max-schrader/cl6lhvrfw001516pkh3s6iv7l",
#             bearing=0,
#             center=go.layout.mapbox.Center(
#                 lat=route_df["lat"].mean(), lon=route_df["lng"].mean()
#             ),
#             pitch=0,
#             zoom=5,
#         ),
#         # margin=go.layout.Margin(l=0, r=0, t=0, b=0),
#     )

#     fig.add_trace(
#         trace
#     )

#     return fig


# for j in range(5):
#     i = 0
#     _r = tt_df.sort_values(('tt_diff', ''), ascending=False).iloc[j]
#     for r_id, r_stops in [[_r.route_1.values[0], (_r[('r1', 'to_id')], _r[('r1', 'from_id')])], [_r.route_2.values[0], (_r[('r2', 'to_id')], _r[('r2', 'from_id')])]]:
#         _df = route_df.loc[(route_df.route_id == r_id) & (route_df.stop_id.isin(r_stops))]
#         if i == 0:
#             res = get_ors_travel_time([_r[("to", "")][::-1], _r[("from", "")][::-1]], [1])
#             print("ORS Estimated TT", res['durations'][0])
#             print("ORS Estimated Distance", res['distances'][0])
#             i += 1
#             fig = make_fig(
#             go.Scattermapbox(
#                 name=r_id,
#                 lat=_df["lat"],
#                 lon=_df["lng"],
#                 mode="markers+text",
#                 marker=dict(
#                     size=10,
#                 ),
#                 text=r_stops,
#                 textfont=dict(
#                     family="sans serif",
#                     size=22,
#                     # weight=5,
#                     color="white"
#                 ),
#             )
#             )
#         print(r_id, _df.departure_datetime.unique()[0], tt_json[r_id][r_stops[0]][r_stops[1]], tt_json[r_id][r_stops[1]][r_stops[0]])
#     fig.show()

#     print("----------------------------------------------------\n\n")



In [49]:
get_ors_travel_time([(47.517331, -122.367252)[::-1], (47.464945, -122.231073)[::-1]], [1])

{'durations': [[1479.18], [0.0]],
 'distances': [[16799.52], [0.0]],
 'destinations': [{'location': [-122.23167, 47.464705],
   'snapped_distance': 52.22}],
 'sources': [{'location': [-122.367296, 47.517332], 'snapped_distance': 3.29},
  {'location': [-122.23167, 47.464705], 'snapped_distance': 52.22}],
 'metadata': {'attribution': 'openrouteservice.org | OpenStreetMap contributors',
  'service': 'matrix',
  'timestamp': 1660686859457,
  'query': {'locations': [[-122.367252, 47.517331], [-122.231073, 47.464945]],
   'profile': 'driving-hgv',
   'responseType': 'json',
   'metricsStrings': ['DISTANCE', 'DURATION'],
   'destinations': ['1'],
   'metrics': ['duration', 'distance']},
  'engine': {'version': '6.7.0',
   'build_date': '2022-02-18T19:37:41Z',
   'graph_date': '2022-08-01T21:12:42Z'}}}